In [1]:
import torch
from torch_geometric.loader import DataLoader
from utils import random_split
from dataset import MyDataset
import torch.nn as nn
from model import Mymodel,Mymodel2
import numpy as np
from sklearn.metrics import mean_squared_error
from metrics import*
from torch_geometric.data import Batch
import pandas as pd
import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem
import os

from GIGN import GIGN

In [2]:
BATCH_SIZE = 1
cutoff = 5
dataset = 'general'

train_pkl = torch.load('./protein_ligand_affinity/data/graph/train_{}A.pkl'.format(cutoff))
valid_pkl = torch.load('./protein_ligand_affinity/data/graph/valid_{}A.pkl'.format(cutoff))
test2016_pkl = torch.load('./protein_ligand_affinity/data/graph/test_{}A.pkl'.format(cutoff))
test2013_pkl = torch.load('./protein_ligand_affinity/data/graph/CASF2013_coreset_{}A.pkl'.format(cutoff))
#csarhiq_pkl = torch.load('./protein_ligand_affinity/data/graph/csar_hiq_{}A.pkl'.format(cutoff))
test2019_pkl = torch.load('./protein_ligand_affinity/data/graph/test2019_{}A.pkl'.format(cutoff))
test2019_id30_pkl = torch.load('./protein_ligand_affinity/data/graph/test2019_id30_{}A.pkl'.format(cutoff))
root = 'protein_ligand_affinity/data/'

train_data =  MyDataset(root, dataset, train_pkl, 'train')
valid_data =  MyDataset(root, dataset, valid_pkl, 'valid')
test2016_data = MyDataset(root, dataset, test2016_pkl, 'test')
test2013_data = MyDataset(root, dataset, test2013_pkl, 'casf2013')
#csarhiq_data = MyDataset(root, dataset, csarhiq_pkl, 'csarhiq')
test2019_data = MyDataset(root, dataset, test2019_pkl, 'test2019')
test2019_id30_data = MyDataset(root, dataset, test2019_id30_pkl, 'test2019_id30')

train_loader = DataLoader(train_data, BATCH_SIZE)
valid_loader = DataLoader(valid_data, BATCH_SIZE)
test2016_loader = DataLoader(test2016_data, BATCH_SIZE)
test2013_loader = DataLoader(test2013_data, BATCH_SIZE)
#csarhiq_loader = DataLoader(csarhiq_data, BATCH_SIZE)
test2019_loader = DataLoader(test2019_data, BATCH_SIZE)
test2019_id30_loader = DataLoader(test2019_id30_data, BATCH_SIZE)

Processed is found...
Processed is found...
Processed is found...
Processed is found...
Processed is found...
Processed is found...


In [67]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Mymodel(atom_in_dim = 35, bond_in_dim = 6, hidden_dim = 256, rbf_num = 9, num_heads = 4, dropout = 0.1)
#model = GIGN(35, 256)
model = model.to(device)

ckpt = 'epoch-164, valid_rmse-1.215379, valid_mae-0.924372, valid_r-0.755567, valid_sd-1.209885.pt'
model.load_state_dict(torch.load(ckpt))

def valid_and_test(model, device, loader):
    model.eval()
    pred_list = []
    label_list = []
    for _, data in enumerate(loader):
        data = data.to(device) 
        label = data.y
        with torch.no_grad():
            pred = model(data)
            pred_list.append(pred.detach().cpu().numpy())
            label_list.append(label.detach().cpu().numpy())
        
    y_pred = np.concatenate(pred_list, axis=0)
    y_true = np.concatenate(label_list, axis=0) 

    return y_pred, y_true

In [68]:
y_pred, y_true = valid_and_test(model, device, test2016_loader)
rmse = RMSE(y_true, y_pred)
mae = MAE(y_true, y_pred)
r = PR(y_true, y_pred)
sd = SD(y_true, y_pred)

In [69]:
print(rmse, mae, r, sd)

1.2449945 0.98465973 0.8237182799916165 1.232818639782146


In [86]:
import numpy as np
mse = [1.4403255, 1.4857787, 1.4172947]
mae = [1.1338453, 1.1845926, 1.1329943]
r = [0.4678340, 0.4362635, 0.4892939]
sd = [1.3715729, 1.3964069, 1.3534201]

In [87]:
print(np.mean(mse), np.std(mse))

1.4477996333333334 0.028453606283601773


In [88]:
print(np.mean(mae), np.std(mae))

1.1504774 0.02412559088782422


In [89]:
print(np.mean(r), np.std(r))

0.46446380000000004 0.02178033542854654


In [90]:
print(np.mean(sd), np.std(sd))

1.3737999666666667 0.01761980160715651


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from scipy.stats import linregress

# 假设 y_true 和 y_pred 已经定义
# 例如:
# y_true = np.random.normal(0, 1, 100)
# y_pred = np.random.normal(0, 1, 100)

# 使用Seaborn绘制散点图和边缘直方图
g = sns.jointplot(x=y_true, y=y_pred, kind="scatter", marginal_kws=dict(bins=25, fill=True, stat='density', color='#FEA55F'), color = '#FFCB5F')

# 计算真实值和预测值的均值和标准差
mu_true, sigma_true = np.mean(y_true), np.std(y_true)
mu_pred, sigma_pred = np.mean(y_pred), np.std(y_pred)

# 生成正态分布的概率密度函数值
x_true = np.linspace(min(y_true), max(y_true), 1000)
y_true_curve = norm.pdf(x_true, mu_true, sigma_true)

x_pred = np.linspace(min(y_pred), max(y_pred), 1000)
y_pred_curve = norm.pdf(x_pred, mu_pred, sigma_pred)

# 绘制正态分布曲线
g.ax_marg_x.plot(x_true, y_true_curve, '#FEA55F')
g.ax_marg_y.plot(y_pred_curve, x_pred, '#FEA55F') 

slope, intercept, r_value, p_value, std_err = linregress(y_true, y_pred)
x_fit = np.linspace(min(y_true), max(y_true), 100)
y_fit = slope * x_fit + intercept

# 在散点图上添加拟合线
g.ax_joint.plot(x_fit, y_fit, '#FEA55F', linewidth=2)

g.ax_joint.text(x=0.05, y=0.95, s="CASF2013\nRMSE=1.365", 
                fontdict={'family': 'Times New Roman', 'size': 16}, ha='left', va='top', transform=g.ax_joint.transAxes)

for label in (g.ax_joint.get_xticklabels() + g.ax_joint.get_yticklabels()):
    label.set_fontname('Times New Roman')
    label.set_fontsize(16)

plt.savefig('plot2013.png', dpi=600) 
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression

# 绘制拟合曲线
model = LinearRegression()
model.fit(y_true.reshape(-1, 1), y_pred)
x_fit = np.linspace(1, 14, 100).reshape(-1, 1)
y_fit = model.predict(x_fit)

plt.figure(dpi=600)  # 设置DPI为600
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.weight'] = 'bold'
# 绘制相关图
plt.figure(figsize=(8, 8))  # 设置图形的大小

plt.scatter(y_true, y_pred, c = '#FF6347', marker='v', zorder = 2, s= 80, edgecolor = '#FF6347',linewidths = 1)  # 绘制散点图   #FAA460, 
plt.plot(x_fit, y_fit, color = '#FF6347', linewidth=3.5)

plt.xlabel('Label', fontname='Times New Roman', fontsize = 28, fontweight='bold')  # 设置x轴标签
plt.ylabel('Prediction', fontname='Times New Roman', fontsize = 28, fontweight='bold')  # 设置y轴标签
plt.tick_params(axis='both', which='major', labelsize=14)

plt.xticks(np.arange(0, 17.1, 2), fontsize=28)
plt.yticks(np.arange(0, 17.1, 2), fontsize=28)

#plt.xlim(0, 16)  # 设置 x 轴的区间
#plt.ylim(0, 16) 
plt.grid(True, linestyle = '--')  # 显示网格线
 
# 设置Y轴的刻度显示格式
plt.text(0.05, 0.92, 'valid', transform=plt.gca().transAxes, fontname='Times New Roman',fontsize=28, fontweight='bold',color='k')
plt.text(0.053, 0.85, 'RMSE = 1.171', transform=plt.gca().transAxes, fontname='Times New Roman',fontsize=18, fontweight='bold',color='k')
plt.text(0.053, 0.79, 'R = 0.776', transform=plt.gca().transAxes, fontname='Times New Roman',fontsize=18, fontweight='bold',color='k')
plt.text(0.9, 0.92, '(A)', transform=plt.gca().transAxes, fontname='Times New Roman',fontsize=32, fontweight='bold',color='k')

plt.savefig('valid.png', dpi=600)
plt.show()  # 显示图形，  基于这段代码修改

In [ ]:
import random
random.seed(42)
random_index = list(range(0, 11904))
random.shuffle(random_index)
index_list = random_index[:256]

In [ ]:
print(index_list)

In [ ]:


class UseHook():
    def __init__(self, model, module):
        self.model = model
        module.register_forward_hook(self.save_hook)

    def save_hook(self, md, fin, fout):
        self.target_feat = fout
        
    def __call__(self, data):
        self.model.eval()
        output = self.model(data)
        return self.target_feat[0].detach().cpu().numpy()

hook = UseHook(model, model.predict_layer.mlp[0])

feats_in_hook  = []
affinity = []

#随机选择256个样本

In [ ]:
for idx in index_list:
    data = Batch.from_data_list([train_data[idx]])
    label = data.y.numpy()

    data = data.to(device)
    feats = hook(data)

    feats_in_hook.append(hook(data))
    affinity.append(label)

feats_in_hook = np.stack(feats_in_hook, axis = 0).reshape(256, 512)
affinity = np.array(affinity).reshape(-1)

x_min, x_max = np.min(affinity), np.max(affinity)
affinity = (affinity - x_min) / (x_max - x_min)
affinity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 假设你有一个包含高维向量的 NumPy 数组，每一行是一个向量
# 这里假设 data 是一个形状为 (n_samples, 256) 的数组
# 你需要将 data 替换为你自己的数据

# 创建一个 t-SNE 模型并将数据降维到 2 维
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity = 30, early_exaggeration = 20, init = 'pca', random_state = 42)
low_dimension = tsne.fit_transform(feats_in_hook)

plt.figure(figsize=(10, 6))
plt.scatter(low_dimension[:, 0], low_dimension[:, 1], marker='o', s = 70, c = affinity, cmap='coolwarm', vmin=0, vmax=1)

# 显示坐标轴边框
plt.gca().xaxis.set_ticks_position('none')  # x轴的刻度线位置设置为'none'
plt.gca().yaxis.set_ticks_position('none')  # y轴的刻度线位置设置为'none'
plt.show()

In [ ]:
'''
t_rmse, t_mae, t_r, t_sd = valid_and_test(model, device, test2016_loader)
        
print("doing final testing!")
print("test_rmse: {:.6f}, test_mae: {:.6f}, test_r: {:.6f}, test_sd: {:.6f}".format(t_rmse, t_mae, t_r, t_sd))
    
t2013_rmse, t2013_mae, t2013_r, t2013_sd = valid_and_test(model, device, test2013_loader)
print("test2013_rmse: {:.6f}, test_mae: {:.6f}, test_r: {:.6f}, test_sd: {:.6f}".format(t2013_rmse, t2013_mae,
                                                                                             t2013_r, t2013_sd))
'''                                    
def valid_and_test(model, device, valid_loader):
    model.eval()
    pred_list = []
    label_list = []
    for _, data in enumerate(valid_loader):
        data = data.to(device) 
        label = data.y
        with torch.no_grad():
            pred = model(data)

    '''
            pred_list.append(pred.detach().cpu().numpy())
            label_list.append(label.detach().cpu().numpy())
        
    y_pred = np.concatenate(pred_list, axis=0)
    y_true = np.concatenate(label_list, axis=0) 

    return y_pred, y_true
    '''
    
    '''
    rmse = RMSE(y_true, y_pred)
    mae = MAE(y_true, y_pred)
    r = PR(y_true, y_pred)

    sd = SD(y_true,y_pred)

    return rmse, mae, r, sd
    '''